In [1]:
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [266]:
from IPython.display import Markdown

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
import seaborn as sns

In [5]:
import geojson
import plotly.express as px
import plotly.graph_objects as go

# <p style = 'text-align:center'> Fonctions </p>

## Features engineering fonctions

In [110]:
def features_choices_15(bench,drop_list,names,info,model):
    bench["age"] = bench["DataYear"]-bench["YearBuilt"]
    bench = bench.drop(columns=drop_list)
    bench = bench.rename(names,axis='columns')
    bench['lat']=bench['loc'].apply(lambda r:r.split('\'')[3]).astype('float')
    bench['lon']=bench['loc'].apply(lambda r:r.split('\'')[7]).astype('float')
    bench['adress']=bench['loc'].apply(lambda r:r.split('\'')[11].split(',')[0].split("\"")[3])
    bench = bench.drop(columns=['loc'])
    bench_info = bench[info]
    bench_model = bench[model]
    return bench_info, bench_model

In [111]:
def features_choices_16(bench,drop_list,names,info,model):
    bench["age"] = bench["DataYear"]-bench["YearBuilt"]
    bench = bench.drop(columns=drop_list)
    bench = bench.rename(names,axis='columns')
    bench_info = bench[info]
    bench_model = bench[model]
    return bench_info, bench_model

In [160]:
def features_choices(bench15_1, bench16_1, to_drop_15, to_drop_16, new_names_15, new_names_16, info_var, model_var):
    bench15_info, bench15_model = features_choices_15(bench15_1, to_drop_15, new_names_15, info_var, model_var)
    bench16_info, bench16_model = features_choices_16(bench16_1, to_drop_16, new_names_16, info_var, model_var)
    bench_info = pd.concat([bench15_info, bench16_info],axis=0).reset_index(drop=True)
#     bench_info = bench_info.drop_duplicates(subset = ["id"]).reset_index(drop=True)
    bench_model = pd.concat([bench15_model, bench16_model],axis=0).reset_index(drop=True)
#     bench_model = bench_model.drop_duplicates(subset = ["id"]).reset_index(drop=True) 
    return bench_info, bench_model

In [161]:
def compute_lput_2(lput):
    lput_2 = 'moderate_energy_use_lput'
    if lput in ['College/University', 'Hospital (General Medical & Surgical)', 'Data Center']:
        lput_2 = 'very_high_energy_use_lput'
    if lput in ['Convention Center','Courthouse','Urgent Care/Clinic/Other Outpatient', 'Laboratory', 'Other - Mall','Parking', 'Other - Entertainment/Public Assembly', 'Lifestyle Center', 'Wholesale Club/Supercenter', 'Medical Office', 'Police Station', 'Hotel', 'Other']:
        lput_2 = 'high_energy_use_lput'
    if lput in ['Supermarket/Grocery Store', 'Office', 'Senior Care Community', 'Other - Restaurant/Bar', 'Restaurant', 'Museum', 'Fitness Center/Health Club/Gym', 'Other/Specialty Hospital', 'Strip Mall', 'Library', 'Retail Store', 'Residence Hall/Dormitory', 'Other - Recreation', 'Other - Education', 'Manufacturing/Industrial Plant', 'K-12 School', 'Refrigerated Warehouse']:
        lput_2 = 'moderate_energy_use_lput'
    if lput in ['Movie Theater', 'Other - Services', 'Social/Meeting Hall', 'Personal Services (Health/Beauty, Dry Cleaning, etc)', 'Distribution Center', 'Automobile Dealership', 'Multifamily Housing', 'Adult Education', 'Other - Utility', 'Financial Office', 'Non-Refrigerated Warehouse', 'Other - Lodging/Residential', 'Performing Arts', 'Pre-school/Daycare', 'Bank Branch', 'Residential Care Facility', 'Worship Facility', 'Repair Services (Vehicle, Shoe, Locksmith, etc)', 'Other - Public Services']:
        lput_2 = 'low_energy_use_lput'
    if lput in ['Prison/Incarceration', 'Self-Storage Facility', 'Outpatient Rehabilitation/Physical Therapy', 'Food Service']:
        lput_2 = 'very_low_energy_use_lput'   
    return lput_2

In [162]:
def compute_sput_2(sput):
    sput_2 = 'moderate_energy_use_sput'
    if sput in ['Data Center', 'Hospital (General Medical & Surgical)', 'Laboratory', 'Hotel', 'College/University', 'Other - Recreation', 'Courthouse', 'Other/Specialty Hospital']:
        sput_2 = 'very_high_energy_use_sput'
    if sput in ['Convenience Store without Gas Station', 'Movie Theater', 'Multifamily Housing', 'Supermarket/Grocery Store', 'Other - Entertainment/Public Assembly', 'Performing Arts', 'Other - Restaurant/Bar']:
        sput_2 = 'high_energy_use_sput'
    if sput in ['Parking', 'Repair Services (Vehicle, Shoe, Locksmith, etc)', 'Adult Education', 'Office', 'Other - Services', 'Restaurant', 'Senior Care Community', 'Manufacturing/Industrial Plant', 'Financial Office', 'Food Sales', 'Retail Store','Aucun']:
        sput_2 = 'moderate_energy_use_sput'
    if sput in ['Other - Public Services', 'Swimming Pool', 'Distribution Center', 'Other', 'Bar/Nightclub', 'Fitness Center/Health Club/Gym', 'Medical Office', 'Non-Refrigerated Warehouse', 'Food Service', 'Self-Storage Facility', 'Bank Branch', 'Worship Facility', 'Vocational School', 'Museum']:
        sput_2 = 'low_energy_use_sput'
    if sput in ['K-12 School', 'Other - Lodging/Residential', 'Other - Education', 'Refrigerated Warehouse', 'Residence Hall/Dormitory']:
        sput_2 = 'very_low_energy_use_sput'   
    return sput_2

In [163]:
def compute_tput_2(tput):
    tput_2 = 'moderate_energy_use_tput'
    if tput in ['Convenience Store without Gas Station', 'Worship Facility', 'Laboratory']:
        tput_2 = 'high_energy_use_tput'
    if tput in ['Fitness Center/Health Club/Gym', 'Data Center', 'Bank Branch', 'Other - Recreation', 'Other - Restaurant/Bar', 'Enclosed Mall', 'Refrigerated Warehouse', 'Supermarket/Grocery Store', 'Financial Office', 'Other', 'Residence Hall/Dormitory', 'K-12 School', 'Multifamily Housing', 'Food Sales', 'Personal Services (Health/Beauty, Dry Cleaning, etc)', 'Medical Office', 'Swimming Pool', 'Parking']:
        tput_2 = 'moderate_energy_use_tput'
    if tput in ['Manufacturing/Industrial Plant', 'Other - Lodging/Residential', 'College/University', 'Food Service', 'Restaurant', 'Other - Services', 'Distribution Center', 'Retail Store', 'Bar/Nightclub', 'Social/Meeting Hall', 'Office', 'Other - Entertainment/Public Assembly', 'Non-Refrigerated Warehouse', 'Strip Mall', 'Aucun', 'Other - Public Services', 'Self-Storage Facility', 'Fast Food Restaurant', 'Pre-school/Daycare', 'Other - Technology/Science']:
        tput_2 = 'low_energy_use_tput'  
    return tput_2

In [164]:
def loc_outlier():
    # from sklearn.neighbors import LocalOutlierFactor
    # X = bench_model_cleaned[['age', 'nb', 'nf', 'building_gfa' ,'parking_gfa','total_gfa', 'lput_gfa', 'sput_gfa', 'tput_gfa', 'energy_use', 'ghg_emission']].values
    # clf = LocalOutlierFactor(n_neighbors=3)
    # a = pd.DataFrame(data=clf.fit_predict(X),columns=['outlier'])
    # a['id']=bench_model_cleaned.id
    # a.outlier.value_counts()
    # bench_model_cleaned = bench_model_cleaned.drop(bench_model_cleaned[bench_model_cleaned.id.isin(a[a['outlier']==-1]['id'].values)].index)
    # clf.negative_outlier_factor_
    return 3

## Cleaning function

In [279]:
def outlier_cleaning(bench,outliers_id):
    bench_no_outlier = bench.drop(bench[bench['id'].isin(outliers_id)].index)
    #(bench_no_outlier['SiteEnergyUse(kBtu)']>=2*10**7)|
    outliers_idx = bench_no_outlier[(bench_no_outlier['energy_use']<=10**4)|(bench_no_outlier.energy_use > 25118864)|(bench_no_outlier.ghg_emission < 1)].index
    bench_no_outlier = bench_no_outlier.drop(outliers_idx)
    return bench_no_outlier

In [166]:
def property_use_types_cleansing_and_imputation(bench_model):
    # Imputation de la variable largest property use type avec la variable primary property type
    unchanged = ["Hotel","Retail Store","Non-Refrigerated Warehouse","Other","Distribution Center","Medical Office","Worship Facility","Laboratory","Refrigerated Warehouse","Supermarket/Grocery Store", "Restaurant", "College/University","Senior Care Community","Self-Storage Facility"]
    for idx in bench_model[(~bench_model["primary_type"].isna())&(bench_model["lput"].isna())].index:
        if (bench_model.loc[idx,"primary_type"]=="Large Office")|(bench_model.loc[idx,"primary_type"]=="Small- and Mid-Sized Office"):
            bench_model.at[idx,"lput"] = "Office"
            bench_model.at[idx,"lput_gfa"] = bench_model.loc[idx,"building_gfa"]    
        if "Multifamily" in bench_model.loc[idx,"primary_type"]:
            bench_model.at[idx,"lput"] = "Multifamily Housing"
            bench_model.at[idx,"lput_gfa"] = bench_model.loc[idx,"building_gfa"]
        if (bench_model.loc[idx,"primary_type"]=="SPS-District K-12"):
            bench_model.at[idx,"lput"] = "K-12 School"
            bench_model.at[idx,"lput_gfa"] = bench_model.loc[idx,"building_gfa"] 
        if (bench_model.loc[idx,"primary_type"]=="Hospital"):
            bench_model.at[idx,"lput"] = "Hospital (General Medical & Surgical)"
            bench_model.at[idx,"lput_gfa"] = bench_model.loc[idx,"building_gfa"]
        if idx in [649, 2262, 2716]:
            bench_model.at[idx,"lput"] = "Retail Store"
            bench_model.at[idx,"lput_gfa"] = bench_model.loc[idx,"building_gfa"]
        if idx==347:
            bench_model.at[idx,"lput"] = "Medical Office"
            bench_model.at[idx,"lput_gfa"] = bench_model.loc[idx,"building_gfa"]
        if idx==309:
            bench_model.at[idx,"lput"] = "Other"
            bench_model.at[idx,"lput_gfa"] = bench_model.loc[idx,"building_gfa"]
        if bench_model.loc[idx,"primary_type"] in unchanged:
            bench_model.at[idx,"lput"] = bench_model.loc[idx,"primary_type"]
            bench_model.at[idx,"lput_gfa"] = bench_model.loc[idx,"building_gfa"]

    # Imputation des second et third property use type
    to_complete_cat = ['lput', 'sput', 'tput']
    for cat in to_complete_cat:
        bench_model[cat] = bench_model[cat].fillna('Aucun')
        bench_model[cat+'_gfa'] = bench_model[cat+'_gfa'].fillna(0)
    
    # Création des nouvelles variables simplifiées lput_1, sput_1 et tput_1
    for var in ['lput','sput','tput']:
        bench_model[var+'_2'] = bench_model[var].apply(compute_lput_2)
    
    # Suppression des données non utilisables 
    drop_idx_1 = bench_model[(bench_model['energy_use'].isna())|(bench_model['ghg_emission'].isna())].index
    bench_model = bench_model.drop(drop_idx_1)
    # Suppression des propriétés dont on ne connais pas le nombre d'étage
    drop_idx_2 = bench_model[(bench_model["nf"].isna())].index
    bench_model = bench_model.drop(drop_idx_2)
    # Regroupement de la classe NonResidential
    bench_model['type']=bench_model['type'].replace('Nonresidential COS','NonResidential')
    # Reset de l'index
    bench_model = bench_model.reset_index(drop=True)
    
    return bench_model

## Data Visualisation Functions

In [167]:
def boxplot(x):
    fig, ax = plt.subplots(figsize=(15,15))
    ax.boxplot(x, vert = False)
    plt.show()

In [168]:
def barplot(x, y, data):
    fig, ax = plt.subplots(figsize=(15,15))
    sns.set_theme(style="whitegrid") 
    ax = sns.barplot(x=x, y=y, data=data)
    plt.show()

In [169]:
def seattle_map():
    f = open(r"C:\Users\Nabil\Desktop\Openclassrooms\Projet4\2016_seattle_city.geojson", "r")
    dep_gjs=geojson.load(f)
    scl = [[0,"rgb(150,0,90)"],[0.125,"rgb(0, 0, 200)"],[0.25,"rgb(0, 25, 255)"],[0.375,"rgb(0, 152, 255)"],[0.5,"rgb(44, 255, 150)"],[0.625,"rgb(151, 255, 0)"],[0.75,"rgb(255, 234, 0)"],[0.875,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]]
    fig = px.choropleth(pd.DataFrame([[1,2],[1,2]],columns=['CD','color']), geojson=dep_gjs, color="color",locations="CD", featureidkey="properties.rowID",projection="mercator",color_continuous_scale=["#ffffff","#eeeeee"])
    fig.update_layout(coloraxis_showscale=False)
    fig.add_traces(go.Scattergeo(lat=bench_info.lat, lon=bench_info.lon, text=bench_model["id"].astype('str')+' : '+bench_model["energy_use"].astype('str'),marker = dict(size = 3,color = bench_model['energy_use'],colorscale = scl, colorbar = dict(titleside = "right", outlinecolor = "rgba(68, 68, 68, 0)"))))
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":20,"l":0,"b":20})
    fig.show()

In [170]:
def histogram(data, x):
    fig, ax = plt.subplots(figsize=(15,15))
    sns.set_theme(style="whitegrid") 
    sns.histplot(data=data, x=x)
    plt.show()

# <p style = 'text-align:center'> Importation des données </p>

In [205]:
bench15_0, bench16_0 = pd.read_csv(r"2015-building-energy-benchmarking.csv"), pd.read_csv(r"2016-building-energy-benchmarking.csv")

In [275]:
outliers_id = np.concatenate((bench15_0[~bench15_0.Outlier.isna()].OSEBuildingID.values,bench16_0[~bench16_0.Outlier.isna()].OSEBuildingID.values))

# <p style = 'text-align:center'> Choix des features </p>

In [241]:
import json
file=open(r"C:\Users\Nabil\Desktop\Openclassrooms\Projet4\socrata_metadata_2015-building-energy-benchmarking.json")
meta_2015=json.load(file)
file_2=open(r"C:\Users\Nabil\Desktop\Openclassrooms\Projet4\socrata_metadata_2016-building-energy-benchmarking.json")
meta_2016=json.load(file_2)

In [288]:
display(Markdown('# <p style=\'text-align:center\'> Description des variables : </p>'))
for i in range(len(meta_2015["columns"])):
    try:
        print('\033[1m' + meta_2015["columns"][i]["name"] + '\033[0m' + ' : ' + meta_2015["columns"][i]["description"]+'\n')
    except:
        continue

# <p style='text-align:center'> Description des variables : </p>

OSEBuildingID : A unique identifier assigned to each property covered by the Seattle Benchmarking Ordinance for tracking and identification purposes.

BuildingType : City of Seattle building type classification.

PrimaryPropertyType : The primary use of a property (e.g. office, retail store). Primary use is defined as a function that accounts for more than 50% of a property. This is the Property Type - EPA Calculated field from Portfolio Manager.

PropertyName : Official or common property name.

TaxParcelIdentificationNumber : 	
Property King County PIN

CouncilDistrictCode : Property City of Seattle council district.

YearBuilt : Year in which a property was constructed or underwent a complete renovation.

PropertyGFATotal : Total building and parking gross floor area.

PropertyGFAParking : Total space in square feet of all types of parking (Fully Enclosed, Partially Enclosed, and Open).

PropertyGFABuilding(s) : Total floor space in square feet between the outside surfaces of a buil

In [207]:
# features à supprimer dans le dataset de 2015 et celui de 2016
to_drop_15 = ["DataYear","YearBuilt",'TaxParcelIdentificationNumber', 'Neighborhood','ListOfAllPropertyUseTypes','YearsENERGYSTARCertified','SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)', 'SourceEUIWN(kBtu/sf)','SourceEUI(kBtu/sf)', 'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 'Electricity(kWh)', 'Electricity(kBtu)', 'NaturalGas(therms)', 'NaturalGas(kBtu)', 'OtherFuelUse(kBtu)','GHGEmissionsIntensity(kgCO2e/ft2)','DefaultData', 'Comment', 'ComplianceStatus', 'Outlier', '2010 Census Tracts', 'Seattle Police Department Micro Community Policing Plan Areas','City Council Districts', 'SPD Beats']
to_drop_16 = ['DataYear', 'City', 'State', 'TaxParcelIdentificationNumber', 'Neighborhood', 'YearBuilt', 'ListOfAllPropertyUseTypes','YearsENERGYSTARCertified','SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)', 'SourceEUIWN(kBtu/sf)','SourceEUI(kBtu/sf)',  'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 'Electricity(kWh)', 'Electricity(kBtu)', 'NaturalGas(therms)', 'NaturalGas(kBtu)', 'DefaultData', 'Comments', 'ComplianceStatus', 'Outlier', 'GHGEmissionsIntensity']

# nouveaux noms pour les features qu'on garde dans le dataset de 2015 et 2016 
new_names_15 = {'OSEBuildingID':'id', 'CouncilDistrictCode':'district_code' , 'BuildingType':'type', 'PrimaryPropertyType':'primary_type', 'PropertyName':'name','Location':'loc', 'NumberofBuildings':'nb', 'NumberofFloors':'nf','PropertyGFATotal':'total_gfa',  'PropertyGFAParking':'parking_gfa', 'PropertyGFABuilding(s)':'building_gfa', 'LargestPropertyUseType':'lput', 'LargestPropertyUseTypeGFA':'lput_gfa', 'SecondLargestPropertyUseType':'sput', 'SecondLargestPropertyUseTypeGFA':'sput_gfa', 'ThirdLargestPropertyUseType':'tput', 'ThirdLargestPropertyUseTypeGFA':'tput_gfa', 'ENERGYSTARScore':'energy_score', 'SiteEnergyUse(kBtu)':'energy_use', 'GHGEmissions(MetricTonsCO2e)':'ghg_emission', 'Zip Codes':'zip'}
new_names_16 = {'OSEBuildingID':'id', 'CouncilDistrictCode':'district_code' ,'BuildingType':'type', 'PrimaryPropertyType':'primary_type', 'PropertyName':'name', 'Address':'adress', 'ZipCode':'zip', 'Latitude':'lat', 'Longitude':'lon', 'NumberofBuildings':'nb', 'NumberofFloors':'nf', 'PropertyGFATotal':'total_gfa', 'PropertyGFAParking':'parking_gfa', 'PropertyGFABuilding(s)':'building_gfa', 'LargestPropertyUseType':'lput', 'LargestPropertyUseTypeGFA':'lput_gfa', 'SecondLargestPropertyUseType':'sput', 'SecondLargestPropertyUseTypeGFA':'sput_gfa', 'ThirdLargestPropertyUseType':'tput', 'ThirdLargestPropertyUseTypeGFA':'tput_gfa', 'ENERGYSTARScore':'energy_score', 'SiteEnergyUse(kBtu)':'energy_use', 'TotalGHGEmissions':'ghg_emission'}

In [208]:
# features pour le dataset des informations des propriétés
info_var = ['id', 'name', 'lat', 'lon', 'adress', 'energy_score', 'zip']

#features pour le dataset des variables du modèle
model_var = ['id', 'age', 'nb', 'nf', 'type', 'primary_type','district_code', 'building_gfa' ,'parking_gfa','total_gfa', 'lput', 'lput_gfa', 'sput', 'sput_gfa', 'tput', 'tput_gfa', 'energy_score', 'energy_use', 'ghg_emission']

In [227]:
bench_info, bench_model = features_choices(bench15_1, bench16_1, to_drop_15, to_drop_16, new_names_15, new_names_16, info_var, model_var)

# <p style = 'text-align:center'> Nettoyage des outliers </p>

In [281]:
# Suppression des outliers
bench_model = outlier_cleaning(bench_model,outliers_id)

# <p style = 'text-align:center'> Valeurs abbérentes </p>

In [229]:
# Propriétés ayant des surfaces négatives 
negativ_area_index = bench_model[(bench_model["total_gfa"]<0)|(bench_model["building_gfa"]<0)|(bench_model["parking_gfa"]<0)].index
for idx in negativ_area_index:
    bench_model.at[idx,"building_gfa"] = np.abs(bench_model.at[idx,"building_gfa"])
    bench_model.at[idx,"parking_gfa"] = np.abs(bench_model.at[idx,"parking_gfa"])
    bench_model.at[idx,"total_gfa"] = np.abs(bench_model.at[idx,"total_gfa"])

In [231]:
# Valeurs abbérentes détecté manuellement
bench_model = bench_model.drop(bench_model[(bench_model.id.isin([33, 34, 54, 60, 61, 70, 85, 100, 102, 113, 230, 49784, 553]))].index)

# <p style = 'text-align:center'> Imputations </p>

In [232]:
bench_model_cleaned = property_use_types_cleansing_and_imputation(bench_model)

In [233]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

Xnum = bench_model_cleaned[['age','nb','nf','building_gfa','parking_gfa','lput_gfa','sput_gfa','tput_gfa','energy_score']].values
Xcat = bench_model_cleaned[['type','district_code','lput','sput','tput']]
enc = OneHotEncoder(handle_unknown='ignore')
Xenc = enc.fit_transform(Xcat.values).toarray()
X = np.concatenate((Xnum, Xenc),axis=1)

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
X_imp = imputer.fit_transform(X)

bench_model_cleaned['energy_score'] = X_imp[:,8]

In [234]:
bench_model_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6287 entries, 0 to 6286
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             6287 non-null   int64  
 1   age            6287 non-null   int64  
 2   nb             6287 non-null   float64
 3   nf             6287 non-null   float64
 4   type           6287 non-null   object 
 5   primary_type   6287 non-null   object 
 6   district_code  6287 non-null   int64  
 7   building_gfa   6287 non-null   int64  
 8   parking_gfa    6287 non-null   int64  
 9   total_gfa      6287 non-null   int64  
 10  lput           6287 non-null   object 
 11  lput_gfa       6287 non-null   float64
 12  sput           6287 non-null   object 
 13  sput_gfa       6287 non-null   float64
 14  tput           6287 non-null   object 
 15  tput_gfa       6287 non-null   float64
 16  energy_score   6287 non-null   float64
 17  energy_use     6287 non-null   float64
 18  ghg_emis

In [235]:
# boxplot(bench_model_cleaned['energy_use'])

In [236]:
# seattle_map()

In [237]:
bench_model_cleaned.to_excel("benchmark_cleaned.xlsx", index=False)

In [124]:
R = bench.energy_use
s = bench.ghg_emission
R_train, R_test, s_train, s_test = train_test_split(R, s, test_size=0.2, random_state=80)
scaler = StandardScaler()
scaler.fit([[x] for x in R_train])
R_train_std =scaler.transform([[x] for x in R_train])
R_test_std = scaler.transform([[x] for x in R_test])

ridge = Ridge(alpha = 10000)
ridge.fit(R_train_std, s_train)
s_pred_train_simple, s_pred_test_simple  = ridge.predict(R_train_std), ridge.predict(R_test_std)

time_simple= get_time(start_time, end_time)

show_metric(s_pred_train_simple, s_train, s_pred_test_simple, s_test, time_simple)

,RMSE_train,RMSE_test,R2_train,R2_test,TIME
valeur,80.679,80.064,0.376,0.371,"(0.0, 0.000999)"


In [126]:
# model_vs_real(s_pred_train_simple, s_train, 'ghg_emission')